# 02 Mlflow homework

In [1]:
import pandas as pd
import numpy as np
print("Pandas version:", pd.__version__)
print("NumPy version:", np.__version__)

Pandas version: 2.2.3
NumPy version: 2.2.6


## question 1 - to get the mlflow version

In [2]:
import mlflow

print("MLflow version:", mlflow.__version__)

MLflow version: 2.22.0


## question 2 - Download and preprocess the data

In [ ]:
# i have downloaded the data into my project folder
# now i will run the cammand stated from the github repo --> preprocess_data.py

# !python "./src/preprocess_data.py" --raw_data_path "./" --dest_path "./output"

In [9]:
# to count the number of files in the output folder and display the file names 
import os

files = os.listdir("./output")
    
# Count the number of files
file_count = len(files)
    
# Print the file names and count
print(f"Number of files in '{"./output"}': {file_count}")
print("Files:")
for file in files:    
    print(file)

Number of files in './output': 4
Files:
dv.pkl
test.pkl
train.pkl
val.pkl


## question 3 - Train a model with autolog

In [ ]:
# now i will run the command stated from the github repo --> train.py

#!python "./src/train.py" --data_path "./output"

In [19]:
# get the min_samples_split and min_samples_leaf from the mlflow tracking server of the latest run

import mlflow

# Get the latest run based on start time
latest_run_df = mlflow.search_runs(order_by=["start_time desc"]).iloc[0]


In [20]:
# Get the run_id of the latest run
run_id = latest_run_df['run_id']

# Get the run details using the run_id
run = mlflow.get_run(run_id)

# Access the parameters from the run's data
min_samples_split = run.data.params.get('min_samples_split', None)

if min_samples_split:
    print(f"Latest min_samples_split: {min_samples_split}")
else:
    print("min_samples_split was not logged in the latest run.")

Latest min_samples_split: 2


## question 4 Launch the tracking server locally

this is done separately and the answer to this question is "--default-artifact-root" and mine is "./artifacts"

In [ ]:
# # Get the latest run based on start time
# latest_run_df = mlflow.search_runs(order_by=["start_time desc"]).iloc[0]

# # Get the run_id of the latest run
# run_id = latest_run_df['run_id']

# # Get the run_id of the active run
# # run_id = mlflow.active_run().info.run_id

# # Get the URI of the logged model
# model_uri = f"runs:/{run_id}/model"

# # Register the model
# mlflow.register_model(model_uri, "random-forest-model")


Successfully registered model 'random-forest-model'.
Created version '1' of model 'random-forest-model'.


<ModelVersion: aliases=[], creation_timestamp=1747828620939, current_stage='None', description=None, last_updated_timestamp=1747828620939, name='random-forest-model', run_id='0e8119c8680246b0a0a06c93ab7c3c32', run_link=None, source='file:///d:/MLOPS%20learning/mlops_zoomcamp/zoomcamp_practice/mlruns/0/0e8119c8680246b0a0a06c93ab7c3c32/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

## question 5 - Tune model hyperparameters


In [ ]:
# this will run a hyperoptimization process to find the best hyperparameters for the model

# !python "./src/hyo.py" --data_path "./output" --num_trials 15


  0%|          | 0/15 [00:00<?, ?trial/s, best loss=?]
                                                      
🏃 View run enchanting-mare-962 at: http://127.0.0.1:5000/#/experiments/164548606967977100/runs/e77108e4b9cb4d67bde6346adc6c5b2a


  0%|          | 0/15 [00:16<?, ?trial/s, best loss=?]
                                                      
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/164548606967977100


  7%|▋         | 1/15 [00:16<03:51, 16.57s/trial, best loss: 5.370086069268862]
                                                                               
🏃 View run puzzled-crow-36 at: http://127.0.0.1:5000/#/experiments/164548606967977100/runs/0f46ba9bfa874766b99b4ab604bf2943


  7%|▋         | 1/15 [00:17<03:51, 16.57s/trial, best loss: 5.370086069268862]
                                                                               
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/164548606967977100


 13%|█▎        | 2/15 [00:17<01:34,  7.30s/tr

In [28]:
# find the best rmse from the mlflow tracking server of the run names random-forest-hyperopt

# Set the tracking URI if needed
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Set the experiment name
experiment_name = "random-forest-hyperopt"

# Search for all runs in the experiment
runs = mlflow.search_runs(experiment_ids=[mlflow.get_experiment_by_name(experiment_name).experiment_id])

# Find the run with the lowest RMSE
best_run = None
best_rmse = float('inf')  # Initialize to a large value

for _, run in runs.iterrows():
    rmse = run['metrics.rmse']  # Access the RMSE metric for the run
    if rmse < best_rmse:
        best_rmse = rmse
        best_run = run

# Print the best RMSE and its associated hyperparameters
if best_run is not None:
    print(f"Best RMSE: {best_rmse}")
else:
    print("No runs found!")


Best RMSE: 5.335419588556921


## question 6 - promote the best model to the model registry

In [31]:
# this will check the top 5 models, then evaluate. and finally it will move the best model to the model registry

!python ./src/register_model.py --data_path ./output --top_n 5


🏃 View run mysterious-grub-768 at: http://127.0.0.1:5000/#/experiments/809024351200110066/runs/b8e86ec1ceed42f3bb1a0da8f7472688
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/809024351200110066
🏃 View run redolent-moose-524 at: http://127.0.0.1:5000/#/experiments/809024351200110066/runs/80c80e27d1884105846dfd8fd8c77f7f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/809024351200110066
🏃 View run melodic-hen-161 at: http://127.0.0.1:5000/#/experiments/809024351200110066/runs/c246acfedb564bd08334270769c5ba4c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/809024351200110066
🏃 View run gregarious-finch-344 at: http://127.0.0.1:5000/#/experiments/809024351200110066/runs/5fdf84fcd9ca411cb277e588c2070d48
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/809024351200110066
🏃 View run monumental-hare-109 at: http://127.0.0.1:5000/#/experiments/809024351200110066/runs/db47be92836f4fa8835aa78e68b29dfd
🧪 View experiment at: http://127.0.0.1:5000/#/experiment